# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
import plotly.express as px

from PIL import Image
from xml.etree import ElementTree
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load Data

In [ ]:
images_dir = "/mnt/hdd/Datasets/Sohas_weapon-Detection/images"
annotation_dir = "/mnt/hdd/Datasets/Sohas_weapon-Detection/annotations/xmls"

In [ ]:
xml_files = glob.glob(f"{annotation_dir}/*.xml")

In [ ]:
filepath = []
xmin = []
xmax = []
ymin = []
ymax = []
labels = []
heights = []
widths = []

for xml_file in xml_files:
    info = ElementTree.parse(xml_file)
    root = info.getroot()
    object = root.find("object")
    file_path = root.find("filename").text
    file_path = os.path.join(images_dir, file_path)
    sizes = root.find("size")
    height = int(sizes.find("height").text)
    width = int(sizes.find("width").text)
    label = 1 if "knife" in object.find("name").text.lower() else 0
    bnd = object.find("bndbox")

    x_min = int(bnd.find("xmin").text)
    x_max = int(bnd.find("xmax").text)
    y_min = int(bnd.find("ymin").text)
    y_max = int(bnd.find("ymax").text)

    filepath.append(file_path)
    heights.append(height)
    labels.append(label)
    widths.append(width)
    xmin.append(x_min)
    xmax.append(x_max)
    ymin.append(y_min)
    ymax.append(y_max)

In [ ]:
df = pd.DataFrame({
    "file_path": filepath, 
    "height": heights,
    "width": widths,
    "xmin": xmin,
    "xmax": xmax,
    "ymin": ymin,
    "ymax": ymax,
    "label": labels
})

In [ ]:
df.head()

In [ ]:
df.label.value_counts()

In [ ]:
df = df[df["label"] == 1].reset_index(drop=True)

In [ ]:
sample_idx = 61
sample_filepath = df.loc[sample_idx]["file_path"]
sample_width = df.loc[sample_idx]["width"]
sample_height = df.loc[sample_idx]["height"]
sample_xmin = df.loc[sample_idx]["xmin"]
sample_xmax = df.loc[sample_idx]["xmax"]
sample_ymin = df.loc[sample_idx]["ymin"]
sample_ymax = df.loc[sample_idx]["ymax"]

img = cv2.imread(sample_filepath)
margin = dict(l=10, r=10, b=10, t=10)
fig = px.imshow(img)
fig.update_layout(width=sample_width, height=sample_height, margin=margin, title=f"File - {sample_filepath.split('/')[-1]}")
fig.add_shape(type="rect", x0=sample_xmin, x1=sample_xmax, y0=sample_ymin, y1=sample_ymax, xref="x", yref="y", line_color="cyan")

# Data Preprocess

In [ ]:
images = df["file_path"].values

In [ ]:
df.loc[0]

In [ ]:
data = []
labels = []

for image_idx in range(len(images)):
    img = load_img(images[image_idx], target_size=(224, 224))
    img_arr = img_to_array(img)
    normalized_img = img_arr / 255.0
    _, h, w, x_min, x_max, y_min, y_max, _ = df.loc[image_idx]
    norm_xmin = x_min / w
    norm_xmax = x_max / w
    norm_ymin = y_min / h
    norm_ymax = y_max / h
    norm_label = (norm_xmin, norm_xmax, norm_ymin, norm_ymax)

    data.append(normalized_img)
    labels.append(norm_label)

In [ ]:
X = np.array(data, dtype=np.float32)
y = np.array(labels, dtype=np.float32)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training

In [ ]:
base_model = InceptionResNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(4, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(loss="mse", optimizer=Adam(learning_rate=1e-4))

In [ ]:
board = EarlyStopping(monitor="val_loss", patience=3)

In [ ]:
history = model.fit(X_train, y_train, batch_size=10, epochs=50, validation_data=(X_test, y_test), callbacks=[board])

In [ ]:
model.save("knife.h5")

# Prediction

In [ ]:
def predict_knife_coords(filepath):
    img = load_img(filepath, target_size=(224, 224))
    img_arr = img_to_array(img)
    normalized_img = img_arr / 255.0

    h, w = Image.open(filepath).height, Image.open(filepath).width
    #print(h, w)
    normalized_img_arr = normalized_img.reshape(1, 224, 224, 3)

    coords = model.predict(normalized_img_arr, verbose=0)

    denorm = np.array([w, w, h, h])
    coords = coords * denorm
    #print(coords)
    coords = coords.astype(np.int32)

    test_xmin, test_xmax, test_ymin, test_ymax = coords[0]
    test_img = load_img(filepath)
    test_img = np.array(test_img)
    roi = test_img[test_xmin:test_xmax, test_ymin:test_ymax]

    image = cv2.imread(filepath)
    margin = dict(l=10, r=10, b=10, t=10)
    fig = px.imshow(image)
    fig.update_layout(width=w, height=h, margin=margin, title=f"{filepath.split('/')[-1]}")
    fig.add_shape(type="rect", x0=test_xmin, x1=test_xmax, y0=test_ymin, y1=test_ymax, xref="x", yref="y", line_color="green")
    fig.show()

In [ ]:
test_file = "/mnt/hdd/Datasets/Sohas_weapon-Detection/images_test/knife_1215.jpg"

In [ ]:
predict_knife_coords(test_file)